In [ ]:
from pyspark.sql import SparkSession
import hail as hl
import os
import time
import dxpy
import logging
import pandas as pd
import re


# Had to set the configuration to navigate RDD partition error
# Build spark
builder = (
    SparkSession
    .builder
    .appName("Autosome QC")  # Set a meaningful application name
    .config("spark.driver.memory", "96g")  # Set driver memory (e.g., 8 GB)
    .config("spark.executor.memory", "108g")  # Set executor memory (e.g., 16 GB)
    .config("spark.executor.cores", "30")  # Optional: Set number of cores per executor 
    .enableHiveSupport()
)
spark = builder.getOrCreate()

hl.init(sc=spark.sparkContext, idempotent=True)


In [ ]:
chr_num = 1

In [ ]:
def repartition_chrm_mt(chr_num):
    db_name = f"exomes"
    # Find database ID of newly created database using dxpy method
    db_uri = dxpy.find_one_data_object(name=f"{db_name}".lower(), classname="database")['id']
    mt_name = f"chr{chr_num}_initial_variant_qc.mt"
    print(mt_name)
    url = f"dnax://{db_uri}/{mt_name}" # Note: the dnax url must follow this format to properly save MT to DNAX
    mt = hl.read_matrix_table(url)
    num_partitions = int(0.2*mt.n_partitions())
    mt = mt.naive_coalesce(num_partitions)
    url = f"dnax://{db_uri}/chr{chr_num}_vqc.mt"
    mt.write(url, overwrite=True)
    return

In [ ]:
repartition_chrm_mt(chr_num)